# GFPGAN Colab Demo (Fork)

This Colab notebook installs dependencies, clones this fork, downloads pretrained weights automatically, and runs the GFPGAN inference script on sample images.

In [ ]:
#@title Install dependencies
import os
import sys
import shutil
import subprocess


def run(cmd, shell=False, check=True):
    print(">>", cmd if shell else " ".join(cmd))
    if shell:
        return subprocess.run(cmd, shell=True, check=check)
    else:
        return subprocess.run(cmd, check=check)

print(sys.version)

# CI smoke: skip heavy installs when NB_CI_SMOKE is set
if os.environ.get("NB_CI_SMOKE"):
    print("NB_CI_SMOKE=1: printing Python version and exiting install cell")
    raise SystemExit(0)

# Upgrade pip tooling quietly
run([sys.executable, "-m", "pip", "install", "--upgrade", "--quiet", "pip", "setuptools", "wheel"])
# Ensure IPython deps are satisfied to avoid resolver warnings
run([sys.executable, "-m", "pip", "install", "--quiet", "-U", "ipython", "jedi"])

# Install torch (GPU wheels if NVIDIA present)
has_nvidia = shutil.which("nvidia-smi") is not None
if has_nvidia:
    print("Detected NVIDIA GPU; installing CUDA wheels")
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch", "torchvision", "torchaudio", "--index-url", "https://download.pytorch.org/whl/cu121"])
else:
    print("No NVIDIA GPU; installing CPU wheels")
    run([sys.executable, "-m", "pip", "install", "--quiet", "torch", "torchvision", "torchaudio"])

# Core deps: match repo constraints (basicsr <= 1.4.2)
run([sys.executable, "-m", "pip", "install", "--quiet", "--no-cache-dir", "--upgrade",
     "basicsr<=1.4.2", "facexlib", "realesrgan", "opencv-python", "tqdm", "numpy", "PyYAML"])

In [ ]:
#@title Clone this fork
import os
import subprocess

REPO_URL = "https://github.com/IAmJonoBo/GFPGAN.git"
REPO_DIR = "GFPGAN"

if not os.path.isdir(REPO_DIR):
    print("Cloning repo...")
    subprocess.run(["git", "clone", "--depth", "1", REPO_URL, REPO_DIR], check=False)
else:
    print("Repo exists, skipping clone.")

print("Repo at:", os.path.abspath(REPO_DIR))
try:
    print("Listing inputs dir:")
    subprocess.run(["ls", "-la", "inputs"], cwd=REPO_DIR, check=False)
except Exception as e:
    print("Note: inputs directory may not exist yet.")

In [ ]:
#@title Run inference on sample images
import os
import sys
import subprocess

repo_dir = "GFPGAN"
cmd = [sys.executable, "inference_gfpgan.py", "-i", "inputs/whole_imgs", "-o", "results", "-v", "1.4", "-s", "2"]
print("Running:", " ".join(cmd))
res = subprocess.run(cmd, cwd=repo_dir, check=False)
print("Exit code:", res.returncode)
print("Results written to ./GFPGAN/results")

In [ ]:
# @title Display a few restored images
import glob
import os
from IPython.display import display, Image

base = "GFPGAN/results/restored_imgs"
imgs = sorted(glob.glob(os.path.join(base, "*")))[:4]
print(f"Displaying {len(imgs)} images from", base)
for p in imgs:
    try:
        display(Image(filename=p))
    except Exception:
        print("Could not display:", p)

In [ ]:
# @title Upload your own images and run GFPGAN
# @markdown Upload one or more images; they will be processed with the settings below.
import os
import shutil
import subprocess
import sys

# Guard: if not in Colab, skip gracefully
IN_COLAB = False
try:
    import google.colab  # type: ignore
    from google.colab import files
    IN_COLAB = True
except Exception:
    pass

upload_dir = "GFPGAN/uploads"
os.makedirs(upload_dir, exist_ok=True)

if IN_COLAB:
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, "wb") as f:
            f.write(data)
        shutil.move(name, os.path.join(upload_dir, os.path.basename(name)))
else:
    print("Not running in Colab; skipping upload UI.")

# Inference parameters
version = "1.4"  # @param ['1', '1.2', '1.3', '1.4']
scale = 2  # @param {type: 'integer'}
only_center_face = False  # @param {type: 'boolean'}
aligned = False  # @param {type: 'boolean'}
autopilot = False  # @param {type: 'boolean'}
hardware_aware = True  # @param {type: 'boolean'}
select_by = "sharpness"  # @param ['sharpness', 'identity']

print("Running inference on uploads...")
cmd = [sys.executable, "inference_gfpgan.py", "-i", "uploads", "-o", "results", "-v", str(version), "-s", str(scale)]
if only_center_face:
    cmd.append("--only_center_face")
if aligned:
    cmd.append("--aligned")
if autopilot:
    cmd.append("--auto")
    cmd.extend(["--select-by", select_by])
if hardware_aware:
    cmd.append("--auto-hw")
subprocess.run(cmd, cwd="GFPGAN", check=False)
print("Done. See GFPGAN/results")

Notes:
- GPU acceleration is optional; uncomment the CUDA wheel index in the install cell to use GPU on Colab.
- The inference script will automatically download the GFPGAN v1.4 weights if not present.
- For large batches, consider enabling the Real-ESRGAN background upsampler by keeping default settings (it is auto-disabled on CPU).